In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import ruptures as rp
from prediction_pipeline.preprocessing import *
from sklearn.linear_model import LinearRegression

/home/hv270/platelets/lib/python3.7/site-packages/FlowCytometryTools/core/docstring.py:56: MatplotlibDeprecationWarning: 
The dedent function was deprecated in Matplotlib 3.1 and will be removed in 3.3. Use inspect.cleandoc instead.
  doc = dedent(doc)


In [2]:
sys_phen = load_sys_phenotypes(data_dir)
sys_phen["time_col"] = pd.to_datetime(sys_phen["Date"],format="%d/%m/%Y")
sys_phen.sort_values("time_col",inplace=True)

602 candidates summary files
Initially 1068 rows, 643 unique IDs
Finally 642 rows, 642 unique IDs


In [4]:
sys_phen.head()

,Nickname,Analyzer ID,Date,Time,Rack,ID,Sample Inf.,Order Type,Reception Date,Measurement Mode,...,IG%/M,NEUT#&/M,NEUT%&/M,NE-SSC/M,NE-SFL/M,NE-FSC/M,IP SUS(RBC)Iron Deficiency?,HCT/M,RDW-CV/M,time_col
840,XN-1000-1-A,XN-20^12829,12/01/2016,13:28:07,6,C0WX8R,B,Initial,12/01/2016 13:25:58,WB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-01-12
844,XN-1000-1-A,XN-20^12829,12/01/2016,10:05:38,2,C0WX14,B,Initial,12/01/2016 10:03:29,WB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-01-12
842,XN-1000-1-A,XN-20^12829,12/01/2016,11:29:19,4,C0WX6V,B,Initial,12/01/2016 11:27:10,WB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-01-12
843,XN-1000-1-A,XN-20^12829,12/01/2016,10:44:17,3,C0WX4Z,B,Initial,12/01/2016 10:42:08,WB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-01-12
841,XN-1000-1-A,XN-20^12829,12/01/2016,12:03:07,5,C0WX7T,B,Initial,12/01/2016 12:00:58,WB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-01-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,XN-1000-1-A,XN-20^12829,28/11/2018,11:49:20,,C1A2PI,B,Manual,28/11/2018 11:47:33,WB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-11-28
515,XN-1000-1-A,XN-20^12829,29/11/2018,13:25:31,,C1A2TA,B,Manual,29/11/2018 13:23:44,WB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-11-29
514,XN-1000-1-A,XN-20^12829,29/11/2018,10:11:21,,C1A2RE,B,Manual,29/11/2018 10:09:34,WB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-11-29
651,XN-1000-1-A,XN-20^12829,30/11/2018,09:09:50,,C1A2X2,B,Manual,30/11/2018 09:08:04,WB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-11-30


In [92]:
int_phen, meta = get_INTERVAL_phenotypes()

Loading INTERVAL phenotypes
(48460, 760)
(45040, 760)
(38832, 760)
(112535, 11)
(102595, 11)
(17768, 772)
(32574, 777)
(32727, 777)
Checked that each Sample has one row max


In [93]:
int_phen["DateTime"] = pd.to_datetime(int_phen["DateTime"])
int_phen["day"] = int_phen["DateTime"].dt.date
int_phen["machine"] = 1+1*(int_phen.Instrument == "XN-10^11041")
int_phen["weekday"] = int_phen["DateTime"].dt.weekday

In [94]:
int_phen["weekday"] = int_phen["DateTime"].dt.weekday
int_phen["yearday"] = int_phen.DateTime.dt.day_of_year
int_phen["hours"] = int_phen.DateTime.dt.hour

In [95]:
exp_by_day_machine = int_phen.groupby(["machine","day"]).SampleNo.nunique()
good_day_machine = exp_by_day_machine.loc[exp_by_day_machine >= 10]
int_phen = int_phen.set_index(["machine","day"])
int_phen = int_phen.loc[int_phen.index.isin(good_day_machine.index.tolist())].reset_index()

In [96]:
int_phen.shape

(30725, 782)

In [97]:
int_phen.head()

,machine,day,FileName,SampleNo,Instrument,DateTime,DateTime_rank,SampleType,WithinSampleRank,EpiCovId_bl,...,HBF_48m,in_cohort,PLT_10_9_L_bl,PDW_fL_bl,MPV_fL_bl,PCT_PCT_bl,IPF_bl,weekday,yearday,hours
0,2,2013-03-01,[XN-10^11041][00-11_8][20130301_101944][ ...,INT_116_6580100,XN-10^11041,2013-03-01 10:19:44,2.0,NHSBT,1,1090916.0,...,NaN,BL,NaN,NaN,NaN,NaN,NaN,4,60,10
1,2,2013-03-01,[XN-10^11041][00-11_8][20130301_102100][ ...,INT_116_6580000,XN-10^11041,2013-03-01 10:21:00,3.0,NHSBT,1,1111731.0,...,NaN,BL,NaN,NaN,NaN,NaN,NaN,4,60,10
2,2,2013-03-01,[XN-10^11041][00-11_8][20130301_102333][ ...,INT_116_4555900,XN-10^11041,2013-03-01 10:23:33,5.0,NHSBT,1,1042358.0,...,NaN,BL,NaN,NaN,NaN,NaN,NaN,4,60,10
3,2,2013-03-01,[XN-10^11041][00-11_8][20130301_102449][ ...,INT_116_4555800,XN-10^11041,2013-03-01 10:24:49,6.0,NHSBT,1,1111391.0,...,NaN,BL,NaN,NaN,NaN,NaN,NaN,4,60,10
4,2,2013-03-01,[XN-10^11041][00-11_8][20130301_102606][ ...,INT_116_4556000,XN-10^11041,2013-03-01 10:26:06,7.0,NHSBT,1,1033359.0,...,0.9,BL,NaN,NaN,NaN,NaN,NaN,4,60,10
